# Multiple regression

**Multiple regression** refers to the use of linear regression with multiple features instead of just one. If there are two features, linear regression learns to model a surface. In higher dimensions with many features, linear regression could represent complex models.

To handle a large number of features, we can utilize the pandas Dataframe. It offers a multidimensional array structure and a variety of features for data analysis. Additionally, a Dataframe can be eaily converted into a NumPy array. To read a CSV file that contains a datset, we can utilize the read_csv() method. Once the dataset is read, we can use the to_numpy() method to convert the data set into a NumPy array.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('perch_full.csv')

# print fist 5 rows of Dataframe
print(df.head(5))

# input data : [length, height, weight]
perch_full = df.to_numpy()

# target data
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0])

# make training and test sets
train_input, test_input, train_target, test_target = train_test_split(
    perch_full, perch_weight, random_state=42)

   length   height   width
0     8.4     2.11    1.41
1    13.7     3.53    2.00
2    15.0     3.82    2.43
3    16.2     4.59    2.63
4    17.4     4.59    2.94


## Feature engineering

From given features $a$ and $b$, we can create a new feature denoted as $a \times b$. This process is known as **feature engineering**.

Sciket-learn offers various classed under the **sklearn.preprocessing** modlue for creating and preprocessing features. These classes such as PolynomialFeatures are called '**transformers**'. Similar to model classed, transformer classes provide like fit() and transform() methods. In order to perform transformation, a training must be done first. The transform() method actually applies the transformation to the date, while the fit() method discovers new combination of features. Note that transformers manipulate and transform input data to create new features or modify the existing ones, without involving target data.

Let's try the **PolynomailFeatures** transformer and apply it to a sample with two features, specifically $2$ and $3$. The 'get_features_names_out()' method shows how the transformed features are generated from combinations of the original features.

In [33]:
from sklearn.preprocessing import PolynomialFeatures

# transformer object
poly = PolynomialFeatures()

# train the transfomer
poly.fit([[2, 3]])

print(poly.transform([[2, 3]]))
print(poly.get_feature_names_out())

[[1. 2. 3. 4. 6. 9.]]
['1' 'x0' 'x1' 'x0^2' 'x0 x1' 'x1^2']


Then we obtain more features. **PolynomialFeatures** creates interaction terms up to the second degree, that is, it adds powered term for each feature and product terms between features. For example, if we have the feature $2$ and $3$, the transformer will add $2^2 = 4$, $3^3=9$, and $2\times 3 = 6$ as additional terms. Additionally, the intercept coefficient $1$ is included, wich is for the the zeroth term, $x_0^0 x_1^0= 1$. This is combinations with repititions, 

$${}_2\mathrm{H}_0 + {}_2\mathrm{H}_1 + {}_2\mathrm{H}_2 = 6.$$

However, in Scikit-learn's linear model, the intercept term is added automatically, so we don't need to creat it manually. If we set **include_bias=False** when using PolynomialFeatures, the transformer will not create an additional feature for the intercept term:

In [36]:
from sklearn.preprocessing import PolynomialFeatures

# transformer object
poly = PolynomialFeatures(include_bias=False)

# train the transfomer
poly.fit([[2, 3]])

print(poly.transform([[2, 3]]))
print(poly.get_feature_names_out())

[[2. 3. 4. 6. 9.]]
['x0' 'x1' 'x0^2' 'x0 x1' 'x1^2']


Now we apply this procedure to our 'train_input'. We save the transformed data from 'train_input' into the array 'train_poly' and check its shape:

In [39]:
from sklearn.preprocessing import PolynomialFeatures

# transformer object
poly = PolynomialFeatures(include_bias=False)

# train the transfomer
poly.fit(train_input) 

# after training the transformer, apply it to transform the input datasets
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)

print("Train input : ", train_input.shape)
print("Train polynomial (by transformer): ", train_poly.shape)
print(poly.get_feature_names_out())

Train input :  (42, 3)
Train polynomial (by transformer):  (42, 9)
['x0' 'x1' 'x2' 'x0^2' 'x0 x1' 'x0 x2' 'x1^2' 'x1 x2' 'x2^2']


Now we train the multiple regression model with 'train_poly' and 'train_target':

In [45]:
from sklearn.linear_model import LinearRegression

# declear the model class
lr = LinearRegression()

# train the model
lr.fit(train_poly, train_target)

# R^2 scores
score_by_training = lr.score(train_poly, train_target)
score = lr.score(test_poly, test_target)
print(f"score : {score_by_training} (training data)")
print(f"score : {score} (test data)")

score : 0.9903183436982124 (training data)
score : 0.9714559911594145 (test data)


We observe that there is no longer an underfitting problem.

## Overfitting

We can incorporate additional features by including higher-order terms such as 3rd powers and 4th powers. The maximum degree of polynomial terms can be defined using the **degree** parameter of the PolynomialFeature class. For instance, setting 'degree=5' implies that the transfomer will generate polynomial terms up to the 5th power. The default value is 'degree=2'.

In [4]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# transformer as a 5th order polynomial 
poly = PolynomialFeatures(degree=5, include_bias=False)

# train the transfomer
poly.fit(train_input)
#print(poly.get_feature_names_out())

# after training the transformer, apply it to transform the input datasets
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)

print("Train input : ", train_input.shape)
print("Train polynomial (by transformer): ", train_poly.shape)

# declear the model class
lr = LinearRegression()

# train the model
lr.fit(train_poly, train_target)

# R^2 scores
score_by_training = lr.score(train_poly, train_target)
score = lr.score(test_poly, test_target)
print(f"score : {score_by_training} (training data)")
print(f"score : {score} (test data)")

Train input :  (42, 3)
Train polynomial (by transformer):  (42, 55)
score : 0.9999999999996176 (training data)
score : -144.40585108215134 (test data)


Here the number of generated features is 55, since

$${}_3\mathrm{H}_1 + {}_3\mathrm{H}_2 + {}_3\mathrm{H}_3 + {}_3\mathrm{H}_4 + {}_3\mathrm{H}_5 = 55.$$

Note that this number is greater than the number of data points in the dataset. As a result, the trained model acheives an almost perfect score on the training data, but the test score is significally lower, indicating a failure to generalize well.

Increasing the number of features can make the linear model more powerful. However, such model tends to overfit the training set and fails to fit the test sets accurately. To address this issue, it appears that we need to reduce the number of features again, in order to find a better balance between model complexity and generalization.